In [1]:
import CAT
import sys
import json
import logging
import numpy as np
import pandas as pd

In [2]:
def setuplogger():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(levelname)s %(asctime)s] %(message)s")
    handler.setFormatter(formatter)
    root.addHandler(handler)

In [3]:
setuplogger()

In [4]:
# choose dataset here
dataset = 'junyi'
# modify config here
# irt
config = {
    'learning_rate': 0.0025,
    'batch_size': 2048,
    'num_epochs': 6,
    'num_dim': 1, # for IRT or MIRT
    'device': 'cuda:0',
    # for NeuralCD
    'prednet_len1': 128,
    'prednet_len2': 64,
}
# #mirt
# config = {
#     'learning_rate': 0.002,
#     'batch_size': 2048,
#     'num_epochs': 10,
#     'num_dim': 10, # for IRT or MIRT
#     'device': 'cpu',
#     # for NeuralCD
#     'prednet_len1': 128,
#     'prednet_len2': 64,
# }

In [5]:
# read datasets
train_triplets = pd.read_csv(f'../data/{dataset}/train_triples.csv', encoding='utf-8').to_records(index=False)
concept_map = json.load(open(f'../data/{dataset}/concept_map.json', 'r'))
concept_map = {int(k):v for k,v in concept_map.items()}
metadata = json.load(open(f'../data/{dataset}/metadata.json', 'r'))

In [6]:
train_data = CAT.dataset.TrainDataset(train_triplets, concept_map,
                                      metadata['num_train_students'], 
                                      metadata['num_questions'], 
                                      metadata['num_concepts'])

In [7]:
# define model here
# model = CAT.model.IRTModel(**config)
model = CAT.model.NCDModel(**config)
# train model
model.init_model(train_data)
model.train(train_data)

[INFO 2022-06-13 16:18:03,352] train on cuda:0
[INFO 2022-06-13 16:18:03,539] Epoch [1] Batch [0]: loss=inf
[INFO 2022-06-13 16:18:05,162] Epoch [1] Batch [20]: loss=0.87573
[INFO 2022-06-13 16:18:06,884] Epoch [1] Batch [40]: loss=0.75021
[INFO 2022-06-13 16:18:08,607] Epoch [1] Batch [60]: loss=0.68374
[INFO 2022-06-13 16:18:10,194] Epoch [1] Batch [80]: loss=0.64783
[INFO 2022-06-13 16:18:11,870] Epoch [1] Batch [100]: loss=0.62631
[INFO 2022-06-13 16:18:13,595] Epoch [1] Batch [120]: loss=0.61272
[INFO 2022-06-13 16:18:15,200] Epoch [1] Batch [140]: loss=0.60218
[INFO 2022-06-13 16:18:16,880] Epoch [1] Batch [160]: loss=0.59401
[INFO 2022-06-13 16:18:18,546] Epoch [1] Batch [180]: loss=0.58797
[INFO 2022-06-13 16:18:20,319] Epoch [1] Batch [200]: loss=0.58246
[INFO 2022-06-13 16:18:21,993] Epoch [1] Batch [220]: loss=0.57804
[INFO 2022-06-13 16:18:23,777] Epoch [1] Batch [240]: loss=0.57402
[INFO 2022-06-13 16:18:25,368] Epoch [1] Batch [260]: loss=0.57019
[INFO 2022-06-13 16:18:27

In [8]:
# save model
model.adaptest_save('../ckpt/irt.pt')